In [1]:
import pandas as pd
import numpy as np
import requests
import os 
from dotenv import load_dotenv
import requests
from tqdm import tqdm
import pickle
import json
pd.options.display.max_columns = None

In [2]:
import spotipy 
from spotipy.oauth2 import SpotifyClientCredentials ## para poner mis credenciales

# Importing pickle file

In [4]:
with open('../data/pickle/dataset_viral150_unique.pickle', 'rb') as tracks:
    tracks = pickle.load(tracks)

In [5]:
tracks.head()

,url,track_clean_new,artist_clean_new
8246,https://open.spotify.com/track/4Km5HrUvYTaSUfi...,bad and boujee (feat. lil uzi vert),migos
8247,https://open.spotify.com/track/343YBumqHu19cGo...,fake love,drake
8248,https://open.spotify.com/track/5aAx2yezTd8zXrk...,starboy,"the weeknd, daft punk"
8249,https://open.spotify.com/track/7BKLCZ1jbUBVqRi...,closer,"the chainsmokers, halsey"
8250,https://open.spotify.com/track/6fujklziTHa8uoM...,black beatles,"rae sremmurd, gucci mane"


In [11]:
tracks.rename(columns={'track_clean_new':'track','artist_clean_new':'artist'}, inplace=True)
tracks.head()

,url,track,artist
8246,https://open.spotify.com/track/4Km5HrUvYTaSUfi...,bad and boujee (feat. lil uzi vert),migos
8247,https://open.spotify.com/track/343YBumqHu19cGo...,fake love,drake
8248,https://open.spotify.com/track/5aAx2yezTd8zXrk...,starboy,"the weeknd, daft punk"
8249,https://open.spotify.com/track/7BKLCZ1jbUBVqRi...,closer,"the chainsmokers, halsey"
8250,https://open.spotify.com/track/6fujklziTHa8uoM...,black beatles,"rae sremmurd, gucci mane"


# spotify
Aquí está la docu: https://spotipy.readthedocs.io/en/2.22.0/#

Configuro el cliente y le metó las keys para acceder a la API de Spotify y poder sacar datos de las canciones que tengo

In [12]:
load_dotenv()

True

In [13]:
id = os.getenv("id")
secret = os.getenv("secret")

client_credentials_manager = SpotifyClientCredentials(client_id=id, client_secret=secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager, requests_timeout=10, retries=10)

Primero voy a sacar el URI de mis canciones. Para ello uso el método de spotipy.search, que permite buscar con una query el nombre del artista y la canción.

In [14]:
## esto es para obtener una uri --> me voy a sacar los uris para mi data frame
artist = 'alan walker'
track = 'alone'
q = 'artist:{} track: {}'.format(artist, track)
results = sp.search(q=q, limit=1, type='track')
print(results['tracks']['items'][0]['uri'])

spotify:track:3LlmKSHR3Rs0Y3KHQLAYDk


In [15]:
for index, row in tracks.iterrows():
    print(row['artist'],'---', row['track'])
    break

migos --- bad and boujee (feat. lil uzi vert)


In [16]:
sp.search(q=q, limit=1, type='track')

{'tracks': {'href': 'https://api.spotify.com/v1/search?query=artist%3Aalan+walker+track%3A+alone&type=track&offset=0&limit=1',
  'items': [{'album': {'album_type': 'album',
     'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/7vk5e3vY1uw9plTHJAMwjN'},
       'href': 'https://api.spotify.com/v1/artists/7vk5e3vY1uw9plTHJAMwjN',
       'id': '7vk5e3vY1uw9plTHJAMwjN',
       'name': 'Alan Walker',
       'type': 'artist',
       'uri': 'spotify:artist:7vk5e3vY1uw9plTHJAMwjN'}],
     'available_markets': ['AD',
      'AE',
      'AG',
      'AL',
      'AM',
      'AO',
      'AR',
      'AT',
      'AU',
      'AZ',
      'BA',
      'BB',
      'BD',
      'BE',
      'BF',
      'BG',
      'BH',
      'BI',
      'BJ',
      'BN',
      'BO',
      'BR',
      'BS',
      'BT',
      'BW',
      'BY',
      'BZ',
      'CA',
      'CD',
      'CG',
      'CH',
      'CI',
      'CL',
      'CM',
      'CO',
      'CR',
      'CV',
      'CW',
      'CY',
     

In [18]:
tqdm.pandas()

uris = []
for index, row in tqdm(tracks.iterrows(), total = tracks.shape[0]):
    q = 'artist:{} track: {}'.format(row['artist'], row['track'])
    res = sp.search(q=q, limit=1, type='track')
    try:
        uri = res['tracks']['items'][0]['uri']
        #print(res['tracks']['items'][0]['uri'])
        uris.append(uri)
    except:
        uris.append('')
    #break

 47%|████▋     | 6843/14659 [11:35<13:14,  9.84it/s]


SpotifyException: http status: 400, code:-1 - https://api.spotify.com/v1/search?q=artist%3Alin-manuel+miranda%2C+alex+lacamoire%2C+ana+villafa%C3%B1e%2C+anthony+ramos%2C+camila+cabello%2C+dessa%2C+ednita+nazario%2C+fat+joe%2C+gilberto+santa+rosa%2C+gina+rodriguez%2C+gloria+estefan%2C+jennifer+lopez%2C+joell+ortiz%2C+john+leguizamo%2C+juan+luis+guerra+4.40%2C+luis+fonsi%2C+marc+anthony%2C+pj+sin+suela%2C+pedro+cap%C3%B3%2C+rita+moreno%2C+rub%C3%A9n+blades%2C+tommy+torres+track%3A+almost+like+praying+%28feat.+artists+for+puerto+rico%29&limit=1&offset=0&type=track:
 Bad request., reason: None

Voy a comprobar que la lista donde me he appendeado todo tiene la misma longitud que mi dataframe, para poder añadir los URIs como una columna

In [ ]:
uris[:3]

In [ ]:
print(len(uris), tracks.shape)

In [ ]:
tracks['URI'] = uris
tracks.head()

Ahora con el método `'audio_features'` de spotipy quiero sacar las propiedades que tienen las canciones. Para ello, tengo que aportar:
- `URI`

Y el resultado de cada una de mis consultas, lo almacenaré en una lista de listas que luego appendearé a mi dataframe.

In [ ]:
sp.audio_features('spotify:track:3OHfY25tqY28d16oZczHc8')

In [ ]:
type(sp.audio_features('spotify:track:3OHfY25tqY28d16oZczHc8'))

In [ ]:
datos_uris = []
for index, row in tracks.iterrows():
    uri = sp.audio_features(row['URI'])
    datos_uris.append(uri)

Y tengo mis datos sobre las canciones, pero tengo que limpiar un poco para poder hacer el append.

In [ ]:
type(pd.DataFrame(datos_uris).iloc[0,0]) ## puedo sacar las claves para nombrar a mis columnas

In [ ]:
columnas = list(pd.DataFrame(datos_uris).iloc[0,0].keys())
columnas[:5]

In [ ]:
df_uris = pd.DataFrame(datos_uris)
df_uris[columnas] = df_uris[0].apply(pd.Series)
df_uris.head()

Lo junto ahora con mi dataframe inicial y limpio las columnas que no necesite.

In [ ]:
tracks = tracks.merge(df_uris, left_on = 'URI', right_on='uri', indicator=True)
tracks['_merge'].value_counts() ## compruebo que el cruce se haga con éxito

In [ ]:
tracks.drop([0, 'uri', '_merge'], axis = 1, inplace=True)
tracks.head()

# Export

In [ ]:
with open('../data/spotify_dataset_spotify.pickle', 'wb') as data_spotify:
    pickle.dump(tracks, data_spotify)